# Semantic Search using the Inference API with the Hugging Face Inference Endpoints Service

TODO [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/elastic/elasticsearch-labs/blob/main/notebooks/search/TODO)


Learn how to use the [Inference API](https://www.elastic.co/guide/en/elasticsearch/reference/current/inference-apis.html) with the Hugging Face Inference Endpoint service for semantic search.

# 🧰 Requirements

For this example, you will need:

- An Elastic deployment:
   - We'll be using [Elastic serverless](https://www.elastic.co/docs/current/serverless) for this example (available with a [free trial](https://cloud.elastic.co/registration?utm_source=github&utm_content=elasticsearch-labs-notebook))

- Elasticsearch 8.14 or above.
   
- A paid [Hugging Face Inference Endpoint](https://huggingface.co/docs/inference-endpoints/guides/create_endpoint) is required to use the Inference API with 
the Hugging Face Inference Endpoint service.

# Create Elastic Cloud deployment or serverless project

If you don't have an Elastic Cloud deployment, sign up [here](https://cloud.elastic.co/registration?utm_source=github&utm_content=elasticsearch-labs-notebook) for a free trial.

# Install packages and connect with Elasticsearch Client

To get started, we'll need to connect to our Elastic deployment using the Python client (version 8.12.0 or above).
Because we're using an Elastic Cloud deployment, we'll use the **Cloud ID** to identify our deployment.

First we need to `pip` install the following packages:

- `elasticsearch`

In [ ]:
!pip install elasticsearch
!pip install datasets
!pip install 'eland[pytorch]'

Next, we need to import the modules we need. 🔐 NOTE: getpass enables us to securely prompt the user for credentials without echoing them to the terminal, or storing it in memory.

In [ ]:
from elasticsearch import Elasticsearch, helpers
from getpass import getpass
import datasets
from eland.ml.pytorch import PyTorchModel
from eland.ml.pytorch.transformers import TransformerModel
from pathlib import Path

Now we can instantiate the Python Elasticsearch client.

First we prompt the user for their password and Cloud ID.
Then we create a `client` object that instantiates an instance of the `Elasticsearch` class.

In [ ]:
# https://www.elastic.co/search-labs/tutorials/install-elasticsearch/elastic-cloud#finding-your-cloud-id
ELASTIC_CLOUD_ID = getpass("Elastic Cloud ID: ")

# https://www.elastic.co/search-labs/tutorials/install-elasticsearch/elastic-cloud#creating-an-api-key
ELASTIC_API_KEY = getpass("Elastic Api Key: ")

# Create the client instance
client = Elasticsearch(
    # For local development
    # hosts=["http://localhost:9200"]
    cloud_id=ELASTIC_CLOUD_ID,
    api_key=ELASTIC_API_KEY,
    request_timeout=120,
    max_retries=10,
    retry_on_timeout=True,
)

### Test the Client
Before you continue, confirm that the client has connected with this test.

In [ ]:
print(client.info())


# define this now so we can use it later
def pretty_search_response(response):
    if len(response["hits"]["hits"]) == 0:
        print("Your search returned no results.")
    else:
        for hit in response["hits"]["hits"]:
            id = hit["_id"]
            score = hit["_score"]
            text = hit["_source"]["text_field"]

            pretty_output = f"\nID: {id}\nScore: {score}\nText: {text}"

            print(pretty_output)

Refer to [the documentation](https://www.elastic.co/guide/en/elasticsearch/client/python-api/current/connecting.html#connect-self-managed-new) to learn how to connect to a self-managed deployment.

Read [this page](https://www.elastic.co/guide/en/elasticsearch/client/python-api/current/connecting.html#connect-self-managed-new) to learn how to connect using API keys.

<a name="create-the-inference-endpoint"></a>
## Create the inference endpoint object

Let's create the inference endpoint by using the [Create inference API](https://www.elastic.co/guide/en/elasticsearch/reference/current/put-inference-api.html).

You'll need an Hugging Face API key (access token) for this that you can find in your Hugging Face account under the [Access Tokens](https://huggingface.co/settings/tokens).

You will also need to have created a [Hugging Face Inference Endpoint service instance](https://huggingface.co/docs/inference-endpoints/guides/create_endpoint) and noted the `url` of your instance. For this notebook, we deployed the `multilingual-e5-small` model.

In [ ]:
API_KEY = getpass("Huggingface API key:  ")
client.inference.put_model(
    inference_id="my_hf_endpoint_object",
    body={
        "service": "hugging_face",
        "service_settings": {
            "api_key": API_KEY,
            "url": "<hf-endpoint-url>",
            "similarity": "dot_product",
        },
    },
    task_type="text_embedding",
)

In [ ]:
client.inference.inference(
    inference_id="my_hf_endpoint_object", input="this is the raw text of my document!"
)

**IMPORTANT:** If you use Elasticsearch 8.12, you must change `inference_id` in the snippet above to `model_id`! 

#

## Create an ingest pipeline with an inference processor

Create an ingest pipeline with an inference processor by using the [`put_pipeline`](https://www.elastic.co/guide/en/elasticsearch/reference/master/put-pipeline-api.html) method. Reference the `inference_id` created above as `model_id` to infer on the data that is being ingested by the pipeline.

In [ ]:
client.ingest.put_pipeline(
    id="hf_pipeline",
    processors=[
        {
            "inference": {
                "model_id": "my_hf_endpoint_object",
                "input_output": {
                    "input_field": "text_field",
                    "output_field": "text_embedding",
                },
            }
        }
    ],
)

Let's note a few important parameters from that API call:

- `inference`: A processor that performs inference using a machine learning model.
- `model_id`: Specifies the ID of the inference endpoint to be used. In this example, the inference ID is set to `my_hf_endpoint_object`. Use the inference ID you defined when created the inference task.
- `input_output`: Specifies input and output fields.
- `input_field`: Field name from which the `dense_vector` representation is created.
- `output_field`:  Field name which contains inference results. 

## Create index

The mapping of the destination index - the index that contains the embeddings that the model will create based on your input text - must be created. The destination index must have a field with the [dense_vector](https://www.elastic.co/guide/en/elasticsearch/reference/current/dense-vector.html) field type to index the output of the model we deployed in Hugging Face (`multilingual-e5-small`).

Let's create an index named `hf-endpoint-index` with the mappings we need.

In [ ]:
client.indices.create(
    index="hf-endpoint-index",
    settings={
        "index": {
            "default_pipeline": "hf_pipeline",
        }
    },
    mappings={
        "properties": {
            "text": {"type": "text"},
            "text_embedding": {
                "type": "dense_vector",
                "dims": 384,
                "similarity": "dot_product",
            },
        }
    },
)

## If you are using Elasticsearch serverless or v8.15+ then you will have access to the new `semantic_text` field
`semantic_text` has significantly faster ingest times and is recommended.

https://github.com/elastic/elasticsearch/blob/main/docs/reference/mapping/types/semantic-text.asciidoc

In [ ]:
client.indices.create(
    index="hf-semantic-text-index",
    mappings={
        "properties": {
            "infer_field": {
                "type": "semantic_text",
                "inference_id": "my_hf_endpoint_object",
            },
            "text_field": {"type": "text", "copy_to": "infer_field"},
        }
    },
)

## Insert Documents

In this example, we want to show the power of using GPUs in Hugging Face's Inference Endpoint service by indexing millions of multilingual documents from the miracl corpus. The speed at which these documents ingest will depend on whether you use a semantic text field (faster) or an ingest pipeline (slower) and will also depend on how much hardware your rent for your Hugging Face inference endpoint. Using a semantic_text field with a single T4 GPU, it may take about 3 hours to index 1 million documents. 

In [ ]:
# langs = [
#     "ar",
#     "bn",
#     "en",
#     "es",
#     "fa",
#     "fi",
#     "fr",
#     "hi",
#     "id",
#     "ja",
#     "ko",
#     "ru",
#     "sw",
#     "te",
#     "th",
#     "zh",
# ]

langs = [
    "en"
]  # for this example, we just use english so that the results are also in english


all_langs_datasets = [
    iter(datasets.load_dataset("miracl/miracl-corpus", lang)["train"]) for lang in langs
]

In [ ]:
MAX_BULK_SIZE = 1000
MAX_BULK_UPLOADS = 1000

sentinel = object()
for j in range(MAX_BULK_UPLOADS):
    documents = []
    while len(documents) < MAX_BULK_SIZE - len(all_langs_datasets):
        for ds in all_langs_datasets:
            text = next(ds, sentinel)
            if text is not sentinel:
                documents.append(
                    {
                        "_index": "hf-semantic-text-index",
                        "_source": {"text_field": text["text"]},
                    }
                )
                # if you are using an ingest pipeline instead of a
                # semantic text field, use this instead:
                # documents.append(
                #     {
                #         "_index": "hf-endpoint-index",
                #         "_source": {"text": text['text']},
                #     }
                # )

    try:
        response = helpers.bulk(client, documents, raise_on_error=False, timeout="60s")
        print(response)
        print("Docs uplaoded:", (j + 1) * MAX_BULK_SIZE)

    except Exception as e:
        print("exception:", str(e))

## Semantic search

After the dataset has been enriched with the embeddings, you can query the data using [semantic search](https://www.elastic.co/guide/en/elasticsearch/reference/current/knn-search.html#knn-semantic-search). Pass a `query_vector_builder` to the k-nearest neighbor (kNN) vector search API, and provide the query text and the model you have used to create the embeddings.

In [ ]:
query = "motor vehicles"
semantic_search_results = client.search(
    index="hf-semantic-text-index",
    query={"semantic": {"field": "infer_field", "query": query}},
)
pretty_search_response(semantic_search_results)

## Reranking with elasticsearch

First, import a reranking model into elasticsearch with eland

In [ ]:
# Load a Hugging Face transformers model directly from the model hub
tm = TransformerModel(
    model_id="cross-encoder/ms-marco-MiniLM-L-6-v2", task_type="text_similarity"
)

# Export the model in a TorchScrpt representation which Elasticsearch uses
tmp_path = "models"
Path(tmp_path).mkdir(parents=True, exist_ok=True)
model_path, config, vocab_path = tm.save(tmp_path)

# Import model into Elasticsearch
ptm = PyTorchModel(client, tm.elasticsearch_model_id())
ptm.import_model(
    model_path=model_path, config_path=None, vocab_path=vocab_path, config=config
)

In [ ]:
client.inference.put(
    task_type="rerank",
    inference_id="es-rerank-endpoint",
    body={
        "service": "elasticsearch",
        "service_settings": {
            "model_id": "cross-encoder__ms-marco-minilm-l-6-v2",
            "num_allocations": 1,
            "num_threads": 1,
        },
        "task_settings": {"return_documents": True},
    },
)

In [ ]:
reranked_search_results = client.search(
    index="hf-semantic-text-index",
    retriever={
        "text_similarity_reranker": {
            "retriever": {
                "standard": {
                    "query": {"semantic": {"field": "infer_field", "query": query}}
                }
            },
            "field": "text_field",
            "inference_id": "es-rerank-endpoint",
            "inference_text": query,
            "rank_window_size": 100,
        }
    },
)
pretty_search_response(reranked_search_results)